<span style="font-size: 5em">🦜</span>

# __LangGraph Essentials__
> **Disclaimer**  
> This notebook uses content adapted from a course-provided notebook available at [github.com/langchain-ai/lca-langgraph-essentials](https://github.com/langchain-ai/lca-langgraph-essentials).  
> All original credit belongs to the course authors.
## Lab 5: Interrupt! Human In The Loop

LangGraph's `interrupt()` function pauses graph execution and waits for human input before continuing. This enables human-in-the-loop workflows where an admin can review unexpected situations and decide how to proceed. Interrupts require a checkpointer to save state between pauses.


In [1]:
from IPython.display import Image, display
import operator
from typing import Annotated, List, Literal, TypedDict
from langgraph.graph import END, START, StateGraph
from langgraph.types import Command, interrupt

In [2]:
from langgraph.checkpoint.memory import InMemorySaver

memory = InMemorySaver()
config = {"configurable": {"thread_id": "1"}}

In [3]:
class State(TypedDict):
    nlist : Annotated[list[str], operator.add]  

In [9]:
def node_a(state: State) -> Command[Literal["b", "c", END]]:
    print("Entered 'a' node")
    select = state["nlist"][-1]
    if select == "b":
        next_node = "b"
    elif select == "c":
        next_node = "c"
    elif select == "q":
        next_node = END
    else: #illegal input, interrupt and ask for next step
        admin = interrupt(f"Unexpected input '{select}'") 
        # The graph will resume from the beginning of the node,
        # the interrupt doesnt hit again because langgraph knows wich interrupt you are on
        print(admin)
        if admin == "continue":
            next_node = "b"
        else:
            next_node = END
            select = "q"
            
    return Command(
        update = State(nlist = [select]),
        goto = next_node
    )


def node_b (state: State) -> State:
    return(State(nlist = ["B"]))

def node_c (state: State) -> State:
    return(State(nlist = ["C"]))

In [10]:
builder = StateGraph(State)

# Add nodes
builder.add_node("a", node_a)
builder.add_node("b", node_b)
builder.add_node("c", node_c)

# Add edges
builder.add_edge(START,"a")
builder.add_edge("b", END)
builder.add_edge("c", END)

# Compile
graph = builder.compile(checkpointer=memory)

In [8]:
while True:
    user = input('b, c, or q to quit: ')
    input_state = State(nlist = [user])
    result = graph.invoke(input_state, config)

    if '__interrupt__' in result:
        print(f"Interrupt:{result}")
        msg = result['__interrupt__'][-1].value
        print(msg)
        human = input(f"\n{msg}: ")

        human_response = Command(
            resume = human
        )
        result = graph.invoke(human_response, config)
        
    if result['nlist'][-1] == "q":
        print("quit")
        break


Entered 'a' node
Entered 'a' node
Entered 'a' node
Entered 'a' node
Interrupt:{'nlist': ['a', 'q', 'b', 'b', 'B', 'c', 'c', 'C', 'a', 'q', 'b', 'b', 'B', 'c', 'c', 'C', 'c', 'c', 'C', 'a'], '__interrupt__': [Interrupt(value="Unexpected input 'a'", id='c3a316341e3f429836dddf2419d2668b')]}
Unexpected input 'a'
Entered 'a' node
continue
Entered 'a' node
Entered 'a' node
Entered 'a' node
Interrupt:{'nlist': ['a', 'q', 'b', 'b', 'B', 'c', 'c', 'C', 'a', 'q', 'b', 'b', 'B', 'c', 'c', 'C', 'c', 'c', 'C', 'a', 'a', 'B', 'b', 'b', 'B', 'c', 'c', 'C', 'f'], '__interrupt__': [Interrupt(value="Unexpected input 'f'", id='7c4f1d896c6e13de544c96761c484563')]}
Unexpected input 'f'
Entered 'a' node
continue
Entered 'a' node
quit


<a id='l5_execution'></a>


## Takeaways

Setup:

- [Node_a uses `interrupt()`](#interrupt_node) to pause execution when unexpected input occurs
- [Checkpointer enables interrupts](#l5_memory) by saving state between pause and resume

Execution:

- When interrupt is called, the [graph pauses](#l5_execution) and waits for human input
- The admin response determines whether to continue execution or end
- Graph state is preserved during the pause and restored when resuming

Try Next:

- Modify the interrupt logic to ask different questions based on the unexpected input
- Add interrupt calls in node_b or node_c to pause at different points in the workflow
